In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

import ipywidgets as widgets
from IPython.display import display

warnings.filterwarnings("ignore")
%matplotlib inline

from voila.configuration import VoilaConfiguration

config = VoilaConfiguration()
config.show_tracebacks = True

## Data analysis

In [2]:
file = pd.read_csv('steps_tracker_dataset.csv')
file

,date,steps,distance_km,calories_burned,active_minutes,sleep_hours,water_intake_liters,mood
0,08/10/2022,4147,3.11,124.41,41,10.5,3.77,stressed
1,11/06/2022,323,0.24,9.69,3,6.0,1.90,sad
2,28/01/2022,6622,4.97,198.66,66,6.0,4.48,sad
3,23/03/2022,11136,8.35,334.08,111,8.0,1.08,tired
4,20/05/2022,2718,2.04,81.54,27,4.9,1.00,tired
...,...,...,...,...,...,...,...,...
495,04/05/2022,17376,13.03,521.28,174,8.0,2.86,energetic
496,16/04/2022,6917,5.19,207.51,69,4.7,0.49,sad
497,08/01/2022,12247,9.19,367.41,122,9.0,0.15,happy
498,01/03/2022,8295,6.22,248.85,83,4.8,2.39,stressed


In [3]:
# Preprocesamiento
file.drop(columns=["date"], inplace=True)  # Eliminamos la fecha, ya que no es útil para el modelo

In [4]:
# Codificar la variable objetivo (mood)
label_encoder = LabelEncoder()
file["mood"] = label_encoder.fit_transform(file["mood"])
file

,steps,distance_km,calories_burned,active_minutes,sleep_hours,water_intake_liters,mood
0,4147,3.11,124.41,41,10.5,3.77,3
1,323,0.24,9.69,3,6.0,1.90,2
2,6622,4.97,198.66,66,6.0,4.48,2
3,11136,8.35,334.08,111,8.0,1.08,4
4,2718,2.04,81.54,27,4.9,1.00,4
...,...,...,...,...,...,...,...
495,17376,13.03,521.28,174,8.0,2.86,0
496,6917,5.19,207.51,69,4.7,0.49,2
497,12247,9.19,367.41,122,9.0,0.15,1
498,8295,6.22,248.85,83,4.8,2.39,3


In [5]:
# Separar características y etiqueta
X = file.drop(columns=["mood"])
y = file["mood"]

In [6]:
X

,steps,distance_km,calories_burned,active_minutes,sleep_hours,water_intake_liters
0,4147,3.11,124.41,41,10.5,3.77
1,323,0.24,9.69,3,6.0,1.90
2,6622,4.97,198.66,66,6.0,4.48
3,11136,8.35,334.08,111,8.0,1.08
4,2718,2.04,81.54,27,4.9,1.00
...,...,...,...,...,...,...
495,17376,13.03,521.28,174,8.0,2.86
496,6917,5.19,207.51,69,4.7,0.49
497,12247,9.19,367.41,122,9.0,0.15
498,8295,6.22,248.85,83,4.8,2.39


In [7]:
y

0      3
1      2
2      2
3      4
4      4
      ..
495    0
496    2
497    1
498    3
499    2
Name: mood, Length: 500, dtype: int64

## Model training

In [8]:
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Normalización
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
# Entrenar el modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

# Model test

In [11]:
# Evaluación
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Accuracy: 0.17
              precision    recall  f1-score   support

   energetic       0.18      0.17      0.17        18
       happy       0.20      0.22      0.21        23
         sad       0.31      0.21      0.25        19
    stressed       0.19      0.18      0.19        22
       tired       0.04      0.06      0.05        18

    accuracy                           0.17       100
   macro avg       0.18      0.17      0.17       100
weighted avg       0.19      0.17      0.18       100



In [1]:
# ---- Permitir entrada de nuevos datos ----

import ipywidgets as widgets
from IPython.display import display
import pandas as pd

# Crear widgets de entrada
steps_widget = widgets.FloatText(description="Pasos caminados:")
distance_widget = widgets.FloatText(description="Distancia (km):")
calories_widget = widgets.FloatText(description="Calorías:")
active_minutes_widget = widgets.IntText(description="Min. activos:")
sleep_hours_widget = widgets.FloatText(description="Horas de sueño:")
water_widget = widgets.FloatText(description="Agua (litros):")
predict_button = widgets.Button(description="Predecir")

# Crear un widget de salida para mostrar la predicción
output_widget = widgets.Output()

# Mostrar widgets
display(steps_widget, distance_widget, calories_widget, active_minutes_widget, sleep_hours_widget, water_widget, predict_button, output_widget)

def predict_mood(_):
    with output_widget:
        output_widget.clear_output()  # Limpiar salida previa
        
        # Capturar datos desde los widgets
        new_data = {
            'steps': steps_widget.value,
            'distance_km': distance_widget.value,
            'calories_burned': calories_widget.value,
            'active_minutes': active_minutes_widget.value,
            'sleep_hours': sleep_hours_widget.value,
            'water_intake_liters': water_widget.value
        }
        
        # Convertir a DataFrame
        df = pd.DataFrame([new_data])
        
        # Normalizar datos (suponiendo que `scaler` ya está definido)
        new_df = scaler.transform(df)
        
        # Predecir estado de ánimo (suponiendo que `model` y `label_encoder` ya están definidos)
        predicted_mood = model.predict(new_df)[0]
        mood_label = label_encoder.inverse_transform([predicted_mood])[0]
        
        # Mostrar el resultado dentro del widget de salida
        print(f"Estado de ánimo estimado: {mood_label}")

# Conectar el botón con la función de predicción
predict_button.on_click(predict_mood)


FloatText(value=0.0, description='Pasos caminados:')

FloatText(value=0.0, description='Distancia (km):')

FloatText(value=0.0, description='Calorías:')

IntText(value=0, description='Min. activos:')

FloatText(value=0.0, description='Horas de sueño:')

FloatText(value=0.0, description='Agua (litros):')

Button(description='Predecir', style=ButtonStyle())

Output()